In [40]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
import string
from stop_words import stop_words

import pickle

We are using the following features
- TfIdf score
- topic frequency
- emoticons

## tfidf



am, is, are, ---> be. 

In [41]:
train_csv_path = '../data/development.csv'
train_df = pd.read_csv(train_csv_path,index_col=0)

In [42]:
train_df

,body,type,body len,body char len,subreddit
95748,Be open-minded and genuinely interested in oth...,ENFJ,138,730,enfj
626910,ENFJs need to realize that to take care of oth...,ENFJ,207,1153,enfj
75943,I know people that are non-pretentious hipster...,ENFJ,213,1264,enfj
335326,My INTJ partner was trying to help me with col...,ENFJ,142,761,intj
1387161,"Personally, I'm not picky. Really any physical...",ENFJ,69,362,INTP
...,...,...,...,...,...
1325016,this is all true. it's an assumption on my par...,ISTP,362,2047,istp
1747454,lol you guys aren't stereotyped as blind to th...,ISTP,59,352,mbti
83834,I'm around women the same way I'm around men. ...,ISTP,191,1017,istp
553224,"&gt;I've lost a hundred pounds, but I did it f...",ISTP,95,518,INTP


In [43]:
def clean(sentence):
    # lemmetize,
    # stopwords
    # handling some words more subtlely
    sentence = sentence.lower().encode('ascii', 'ignore').decode()

    sentence = ' '.join([word for word in sentence.split(' ') if word not in stop_words])
    sentence = re.sub("@\S+", " ", sentence)
    sentence = re.sub("https*\S+", " ", sentence)
    sentence = re.sub("#\S+", " ", sentence)
    sentence = re.sub("\'\w+", '', sentence)
    sentence = re.sub('[%s]' % re.escape(string.punctuation), ' ', sentence)
    sentence= re.sub(r'\w*\d+\w*', '', sentence)
    sentence = re.sub('\s{2,}', " ", sentence)
    return sentence
    
train_df['body'] = train_df['body'].apply(clean)
train_df['body']

95748      open minded genuinely interested people think ...
626910     enfjs need realize take care others need take ...
75943      i know people non pretentious hipsters exist w...
335326     intj partner trying help college algebra one t...
1387161    personally i picky really physical verbal affe...
                                 ...                        
1325016    true assumption part assume individuals field ...
1747454    lol you guys stereotyped blind damage you infl...
83834      i around women way i around men i sense level ...
553224      gt i lost hundred pounds but i me this truth ...
593954      gt introverts you see us extroverts get energ...
Name: body, Length: 1600, dtype: object

In [44]:
for sentence in train_df['body']:
    print(sentence)
    break

open minded genuinely interested people think things like i already enough friends people life close off take initiative wait people talk you first call you first etc as moving new city try checking meetups similar interests joining clubs volunteering taking classes go gym sit around cafe hang park anywhere others be you know you could even post ad area asking someone tour guide one friends tour guide girl new city getting paid anything completely platonic but fun go explore new city 


In [45]:
vectorizer = CountVectorizer()  
x = vectorizer.fit_transform(train_df['body'])

In [46]:
tfidf_model = TfidfTransformer()  
tfidf = tfidf_model.fit_transform(x)  


## Topic Selection

In [47]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train', shuffle=True)
newsgroups_test = fetch_20newsgroups(subset="test", shuffle = True)

In [48]:
list(newsgroups_train.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [49]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [50]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/baihaocheng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [58]:
processed_docs = []

for doc in newsgroups_train.data:
    processed_docs.append(preprocess(doc))

[['lerxst',
  'thing',
  'subject',
  'nntp',
  'post',
  'host',
  'organ',
  'univers',
  'maryland',
  'colleg',
  'park',
  'line',
  'wonder',
  'enlighten',
  'door',
  'sport',
  'look',
  'late',
  'earli',
  'call',
  'bricklin',
  'door',
  'small',
  'addit',
  'bumper',
  'separ',
  'rest',
  'bodi',
  'know',
  'tellm',
  'model',
  'engin',
  'spec',
  'year',
  'product',
  'histori',
  'info',
  'funki',
  'look',
  'mail',
  'thank',
  'bring',
  'neighborhood',
  'lerxst'],
 ['guykuo',
  'carson',
  'washington',
  'subject',
  'clock',
  'poll',
  'final',
  'summari',
  'final',
  'clock',
  'report',
  'keyword',
  'acceler',
  'clock',
  'upgrad',
  'articl',
  'shelley',
  'qvfo',
  'innc',
  'organ',
  'univers',
  'washington',
  'line',
  'nntp',
  'post',
  'host',
  'carson',
  'washington',
  'fair',
  'number',
  'brave',
  'soul',
  'upgrad',
  'clock',
  'oscil',
  'share',
  'experi',
  'poll',
  'send',
  'brief',
  'messag',
  'detail',
  'experi',
  

In [59]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

In [60]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [61]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [52]:
class features:
    def __init__(self) -> None:
        pass

In [62]:
feature = features()
feature.vect = vectorizer
feature.tfidf = tfidf_model
feature.topic = lda_model

In [54]:
with open('../feature_extraction.obj','wb') as f:
    pickle.dump(feature,f)

In [55]:
with open('../feature_extraction.obj','rb') as f:
    pickle.load(f)